In [2]:
import pickle
import numpy as np

pncdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/PNC/'
pncdemo = pickle.load(open(f'{pncdir}/demographics.pkl', 'rb'))
pnctopdir = '/home/anton/Documents/Tulane/Research/Work/ContrastiveLearning/PNC/'

task = 'emoid'

fc = []
aps20 = []
aps15 = []
aps10 = []
aps5 = []
aps3 = []
aps1 = []
age = []
sex = []
race = []

for sub in pncdemo['age_at_cnb']:
    try:
        a = pncdemo['age_at_cnb'][sub]
        s = pncdemo['Sex'][sub]
        r = pncdemo['Race'][sub]
        if r not in ['AA', 'EA']:
            continue
        s = s == 'M'
        r = r == 'AA'
        fc.append(np.load(f'{pncdir}/fc/{sub}_task-{task}_fc.npy'))
        aps20.append(np.load(f'{pnctopdir}/Top20/{sub}_task-{task}top20_fc.npy'))
        aps3.append(np.load(f'{pnctopdir}/Top3/{sub}_task-{task}top3_fc.npy'))
        aps1.append(np.load(f'{pnctopdir}/Top1/{sub}_task-{task}top1_fc.npy'))
        age.append(a)
        sex.append(s)
        race.append(r)
    except:
        pass

age = np.stack(age)
sex = np.stack(sex).astype('int')
race = np.stack(race).astype('int')
fc = np.stack(fc)
aps20 = np.stack(aps20)
aps1 = np.stack(aps1)

print([x.shape for x in [age, sex, race, fc, aps20, aps1]])

[(1329,), (1329,), (1329,), (1329, 34716), (1329, 34716), (1329, 34716)]


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, LogisticRegression

class MLP(nn.Module):
    def __init__(self, out):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(34716,30).float().cuda()
        self.fc2 = nn.Linear(30,out).float().cuda()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = x.squeeze()
        return x

class GCN(nn.Module):
    def __init__(self, out):
        super(GCN, self).__init__()
        self.fc1 = nn.Linear(34716,30).float().cuda()
        self.fc2 = nn.Linear(30,out).float().cuda()

    def forward(self, x, E):
        x = F.relu(self.fc1(x))
        x = x.squeeze()
        x = E@x
        x = self.fc2(x)
        x = x.squeeze()
        return x

def rmse(yhat, y):
    return np.mean((yhat-y)**2)**0.5

def rmse2(yhat, y):
    return torch.mean((yhat-y)**2)**0.5

accs1 = []
accs2 = []
accs3 = []

for _ in range(10):

    xtr, xt, ytr, yt, vtr, vt, wtr, wt = train_test_split(fc, age, aps20, aps1, train_size=0.8)

    # rega = Ridge(alpha=1).fit(xtr, ytr)
    # regb = Ridge(alpha=1).fit(xtr-wtr, ytr)
    # # rega = LogisticRegression(max_iter=1000).fit(xtr, ytr)
    # # regb = LogisticRegression(max_iter=1000).fit(xtr-wtr, ytr)

    # yhata = rega.predict(xt)
    # yhatb = regb.predict(xt-wt)
    # # yhata = rega.predict_proba(xt)
    # # yhatb = regb.predict_proba(xt-wt)

    # yhat = (yhata+yhatb)/2
    # acc = rmse(yhat, yt)
    # # yhat = np.argmax(yhat, axis=1)
    # # acc = np.mean(yhat == yt)
    # accs1.append(acc)
    # print(acc)

    mlp = MLP(1)
    gcn = GCN(1)

    opa = torch.optim.Adam(mlp.parameters(), lr=1e-4, weight_decay=1e-4)
    opb = torch.optim.Adam(gcn.parameters(), lr=1e-4, weight_decay=1e-4)

    Ntr = len(xtr)
    Nt = len(xt)
    
    Etr = torch.ones(Ntr, Ntr)/Ntr + torch.eye(Ntr)
    Etr = Etr.float().cuda()
    
    Et = torch.ones(Ntr+Nt, Ntr+Nt)/(Ntr+Nt) + torch.eye(Ntr+Nt)
    Et = Et.float().cuda()

    nepochs = 5000
    pperiod = 500
    ce = nn.CrossEntropyLoss()
    
    xtr = torch.from_numpy(xtr).float().cuda()
    ytr = torch.from_numpy(ytr).long().cuda()
    xt = torch.from_numpy(xt).float().cuda()
    yt = torch.from_numpy(yt).long().cuda()

    for e in range(nepochs):
        opa.zero_grad()
        opb.zero_grad()
        yhata = mlp(xtr)
        yhatb = gcn(xtr, Etr)
        lossa = rmse2(yhata, ytr)
        lossb = rmse2(yhatb, ytr)
        # lossa = ce(yhata, ytr)
        # lossb = ce(yhatb, ytr)
        lossa.backward()
        lossb.backward()
        opa.step()
        opb.step()
        if e % pperiod == 0 or e == nepochs-1:
            print(f'{e} {float(lossa)} {float(lossb)}')

    with torch.no_grad():
        yhata = mlp(xt)
        yhatb = gcn(torch.cat([xtr, xt], axis=0), Et)[Ntr:]
        acca = float(rmse2(yhata, yt))
        accb = float(rmse2(yhatb, yt))
        # yhata = torch.argmax(yhata, dim=1).detach().cpu().numpy()
        # yhatb = torch.argmax(yhatb, dim=1).detach().cpu().numpy()
        # acca = np.mean(yhata == yt.detach().cpu().numpy())
        # accb = np.mean(yhatb == yt.detach().cpu().numpy())
        print(acca)
        print(accb)
        accs2.append(acca)
        accs3.append(accb)

print('---')
# print(np.mean(accs1), np.std(accs1))
print(np.mean(accs2), np.std(accs2))
print(np.mean(accs3), np.std(accs3))

0 14.614675521850586 14.611854553222656
500 1.1868321895599365 2.067201614379883
1000 0.21831853687763214 0.9813175201416016
1500 0.10926540940999985 0.46686476469039917
2000 0.08424967527389526 0.27859917283058167
2500 0.07564762979745865 0.19285067915916443
3000 0.07312721014022827 0.15111427009105682
3500 0.0742514505982399 0.12591402232646942
4000 0.07624126970767975 0.11299039423465729
4500 0.07730118930339813 0.1056116595864296
4999 0.05518325790762901 0.08887757360935211
2.8701558113098145
2.642207384109497
0 14.798666954040527 14.612897872924805
500 1.6970840692520142 1.8030000925064087
1000 0.24957388639450073 0.7646662592887878
1500 0.09078259021043777 0.4263554811477661
2000 0.06076277047395706 0.2745044529438019
2500 0.05025229975581169 0.22119274735450745
3000 0.04338376969099045 0.1780514419078827
3500 0.036099329590797424 0.1568181961774826
4000 0.028447171673178673 0.14860761165618896
4500 0.023549148812890053 0.1449964940547943
4999 0.07064847648143768 0.10766331106424